In [3]:
!rm *.csv
!rm cnn*
!wget http://smartheatmap.ddns.net/dist/p.csv
!wget http://smartheatmap.ddns.net/dist/n.csv
!wget http://smartheatmap.ddns.net/dist/w2v.zip
!unzip w2v.zip

rm: cannot remove '*.csv': No such file or directory
rm: cannot remove 'cnn*': No such file or directory
--2019-07-18 15:55:20--  http://smartheatmap.ddns.net/dist/p.csv
Resolving smartheatmap.ddns.net (smartheatmap.ddns.net)... 188.120.231.51
Connecting to smartheatmap.ddns.net (smartheatmap.ddns.net)|188.120.231.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27630235 (26M) [text/csv]
Saving to: ‘p.csv’

p.csv               100%[===================>]  26.35M  7.32MB/s    in 3.6s    

2019-07-18 15:55:23 (7.32 MB/s) - ‘p.csv’ saved [27630235/27630235]

--2019-07-18 15:55:25--  http://smartheatmap.ddns.net/dist/n.csv
Resolving smartheatmap.ddns.net (smartheatmap.ddns.net)... 188.120.231.51
Connecting to smartheatmap.ddns.net (smartheatmap.ddns.net)|188.120.231.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26037206 (25M) [text/csv]
Saving to: ‘n.csv’

n.csv               100%[===================>]  24.83M  8.32MB/s    in 3.0s    

In [0]:
import pandas as pd
import numpy as np

data_positive = pd.read_csv('p.csv', sep=';', error_bad_lines=False, usecols=['ttext'])
data_negative = pd.read_csv('n.csv', sep=';', error_bad_lines=False, usecols=['ttext'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['ttext'].values[:sample_size],
                           data_negative['ttext'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [0]:
import re

def clear_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [clear_text(t) for t in raw_data]


In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

SENTENCE_LENGTH = 26
NUM = 100000

tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

def to_seq(x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
x_train = to_seq(x_train)
x_test = to_seq(x_test)

Using TensorFlow backend.


In [8]:
from gensim.models import Word2Vec

wvmodel = Word2Vec.load("w2v/tweets_model.w2v")

DIM = wvmodel.vector_size 
embedding_matrix = np.zeros((NUM, DIM))

for word, i in tokenizer.word_index.items():
    if i >= NUM:
        break
    if word in wvmodel.wv.vocab.keys():
        embedding_matrix[i] = wvmodel.wv[word]

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
from keras.layers import Input
from keras.layers.embeddings import Embedding

m_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
# m_embed = Embedding(NUM, 
#                     DIM, 
#                     input_length=SENTENCE_LENGTH,
#                     weights=[embedding_matrix], 
#                     trainable=True)(m_input)
DIM=300
m_embed = Embedding(NUM, 
                    DIM, 
                    input_length=SENTENCE_LENGTH)(m_input)

W0718 15:58:39.197914 140439064582016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 15:58:39.233949 140439064582016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 15:58:39.240551 140439064582016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [10]:
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout, LSTM, CuDNNLSTM, TimeDistributed
from keras.models import Model, Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.regularizers import l1

# x = Dropout(0.2)(m_embed)

# conv_branches = []
# for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
#     for i in range(filters_count):
#         i = Conv1D(filters=1, 
#                    kernel_size=size, 
#                    padding='valid', 
#                    activation='relu')(x)
#         i = GlobalMaxPooling1D()(i)
#         conv_branches.append(i)

# x = concatenate(conv_branches, 
#                 axis=1)

# x = Dropout(0.2)(x)
# x = Dense(30, activation='relu')(x)
# x = Dense(1)(x)

# m_output = Activation('sigmoid')(x)

model = Sequential()
model.add(Embedding(NUM, 
                    DIM, 
                    input_length=SENTENCE_LENGTH))

model.add(CuDNNLSTM(units=DIM, return_sequences=True))  
model.add(Dropout(0.2))

model.add(CuDNNLSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(CuDNNLSTM(units=50))  
model.add(Dropout(0.2))  

model.add(Dense(1, activation='sigmoid'))

# model = Model(inputs=[m_input], 
#               outputs=[m_output])

W0718 15:58:41.000008 140439064582016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0718 15:58:41.011883 140439064582016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras import backend as K


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [12]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy',
                       precision, 
                       recall, 
                       f1])

W0718 15:58:41.339756 140439064582016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 15:58:41.362867 140439064582016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0718 15:58:41.369194 140439064582016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("cnn-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', 
                             save_best_only=True, 
                             mode='max', 
                             period=1)

history = model.fit(x_train, 
                    y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_split=0.25, 
                    callbacks = [checkpoint])

Train on 134307 samples, validate on 44769 samples
Epoch 1/10
134307/134307 [==============================] - 147s 1ms/step - loss: 0.4855 - acc: 0.7579 - precision: 0.7622 - recall: 0.7517 - f1: 0.7479 - val_loss: 0.4485 - val_acc: 0.7828 - val_precision: 0.7806 - val_recall: 0.7860 - val_f1: 0.7780
Epoch 2/10
134307/134307 [==============================] - 139s 1ms/step - loss: 0.2960 - acc: 0.8737 - precision: 0.8790 - recall: 0.8665 - f1: 0.8688 - val_loss: 0.4946 - val_acc: 0.7736 - val_precision: 0.7852 - val_recall: 0.7528 - val_f1: 0.7629
Epoch 3/10
134307/134307 [==============================] - 140s 1ms/step - loss: 0.1479 - acc: 0.9412 - precision: 0.9457 - recall: 0.9360 - f1: 0.9388 - val_loss: 0.7355 - val_acc: 0.7608 - val_precision: 0.7549 - val_recall: 0.7732 - val_f1: 0.7580
Epoch 4/10
134307/134307 [==============================] - 139s 1ms/step - loss: 0.0790 - acc: 0.9689 - precision: 0.9718 - recall: 0.9657 - f1: 0.9678 - val_loss: 1.0654 - val_acc: 0.7573 - v

In [14]:
model.predict(to_seq(["Зачем мне подрывать и без того не самую безупречную репутацию кураторов?",
                       "сломанная лавочка",
                       "Ну это же логично, что я дождался, пока ты прочтешь и удалил",
                       "трансгендер на улице",
                       "США",
                       "СССР",
                       "фашист",
                       "на улице поставили хреновую красивую скамейку",
                       "на улице хрено поставили красивую скамейку",
                       "на улице Ленина поставили хреновую скамейку",
                       "красивую скамейку",
                       "мне нравится эта хрень, но будет нереально круто",
                       "Жить конечно тут наверное нет",
                       "Очень-очень красиво, сказочно, солнечно, жарко, волшебно....можно много слов написать, и все они будут про это не забываемое место. Я  не знаю у кого как. Но я очень люблю Сочи-Адлер и люблю тут отдыхать с детьми и одна и с подругами и вообще. Жить конечно тут наверное нет. Но летом сюда-это точно. На все лето, сюда☺☺☺🤩🤩🤩🤩🤩",
                       "эта скамейка очень красивая",
                       "сегодня было грустно",
                       "эта скамейка полная чушь"]))

array([[5.6044459e-03],
       [4.3852100e-01],
       [8.7648630e-05],
       [6.1606467e-03],
       [1.5796575e-01],
       [5.0603300e-01],
       [8.3029270e-04],
       [9.9909830e-01],
       [9.9909830e-01],
       [9.7110498e-01],
       [4.2142719e-02],
       [9.9966228e-01],
       [2.0614266e-04],
       [1.0982230e-01],
       [9.4761878e-02],
       [5.1975250e-05],
       [2.7257418e-05]], dtype=float32)